In [31]:
import pandas as pd

In [32]:
train_df = pd.read_csv('sentences/train.csv',encoding='latin1')
test_df = pd.read_csv('sentences/test.csv',encoding='latin1')
print(train_df.head(10))


       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   
5  28b57f3990  http://www.dothebouncy.com/smf - some shameles...   
6  6e0c6d75b1  2am feedings for the baby are fun when he is a...   
7  50e14c0bb8                                         Soooo high   
8  e050245fbd                                        Both of you   
9  fc2cbefa9d   Journey!? Wow... u just became cooler.  hehe....   

                                       selected_text sentiment Time of Tweet  \
0                I`d have responded, if I were going   neutral       morning   
1                                           Sooo SAD  negative          noon   
2          

In [33]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_sentiment=OneHotEncoder()
train_Y=onehot_encoder_sentiment.fit_transform(train_df[['sentiment']]).toarray()
test_Y=onehot_encoder_sentiment.fit_transform(test_df[['sentiment']]).toarray()

print(train_Y)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [34]:
train_Y_df = pd.DataFrame(train_Y)



In [35]:
train_df['selected_text'] = train_df['selected_text'].fillna('').astype(str)
train_df['text'] = train_df['text'].fillna('').astype(str)

test_df['selected_text'] = train_df['selected_text'].fillna('').astype(str)
test_df['text'] = train_df['text'].fillna('').astype(str)


In [67]:

from tensorflow.keras.preprocessing.text import Tokenizer
import json

from tensorflow.keras.utils import pad_sequences

## Define the vocabulary size
voc_size=10000

max_sentence_length=128
tokenizer = Tokenizer(num_words=voc_size, oov_token="<OOV>")  # Limit vocabulary size

tokenizer.fit_on_texts(train_df['selected_text'])  # Fit tokenizer on training data

# Encode the training and test data
encoded_train = tokenizer.texts_to_sequences(train_df['selected_text'])
encoded_test = tokenizer.texts_to_sequences(test_df['selected_text'])

train_X=pad_sequences(encoded_train,padding='pre',maxlen=max_sentence_length)
test_X =pad_sequences(encoded_test,padding='pre',maxlen=max_sentence_length)
word_index = tokenizer.word_index

# Save word index
with open('word_index.json', 'w') as f:
    json.dump(word_index, f)
print(train_X[1])
print(len(train_X[1]))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 420  72]
128


In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=128, input_length=128))  # Embedding layer
model.add(LSTM(128, return_sequences=False))  # LSTM layer
model.add(Dense(3, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 128, 128)          1280000   
                                                                 
 lstm_4 (LSTM)               (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1411971 (5.39 MB)
Trainable params: 1411971 (5.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [69]:
## Create an instance of EarlyStoppping Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [70]:
## Train the model with early sstopping
history=model.fit(
    train_X,train_Y,epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping]
)

Epoch 1/10
687/687 [==============================] - 43s 60ms/step - loss: 0.5539 - accuracy: 0.7770 - val_loss: 0.4423 - val_accuracy: 0.8361
Epoch 2/10
687/687 [==============================] - 46s 67ms/step - loss: 0.3408 - accuracy: 0.8765 - val_loss: 0.4271 - val_accuracy: 0.8432
Epoch 3/10
687/687 [==============================] - 51s 74ms/step - loss: 0.2595 - accuracy: 0.9073 - val_loss: 0.4651 - val_accuracy: 0.8268
Epoch 4/10
687/687 [==============================] - 53s 77ms/step - loss: 0.2131 - accuracy: 0.9239 - val_loss: 0.5061 - val_accuracy: 0.8295
Epoch 5/10
687/687 [==============================] - 52s 76ms/step - loss: 0.1729 - accuracy: 0.9390 - val_loss: 0.5345 - val_accuracy: 0.8288
Epoch 6/10
687/687 [==============================] - 55s 80ms/step - loss: 0.1402 - accuracy: 0.9493 - val_loss: 0.6101 - val_accuracy: 0.8266
Epoch 7/10
687/687 [==============================] - 56s 82ms/step - loss: 0.1149 - accuracy: 0.9580 - val_loss: 0.6421 - val_accuracy:

In [71]:
## Save model file
model.save('LSTM_sentiment_analysis.h5')

c:\Users\noams\Desktop\MyProjects\SentimentAnalysis\myEnv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [79]:
import numpy as np
def sentiment_predict(sentence):
    categories_map = {0:'negative' , 1:'neutral' , 2:'positive'}
    
    myEncoder = tokenizer.texts_to_sequences([sentence])
    print(myEncoder)
    padded_input=pad_sequences(myEncoder,padding='pre',maxlen=max_sentence_length)
    #print(padded_input)
    prediction=model.predict(padded_input)
    print(prediction)
    print('The sentiment is',categories_map[np.argmax(prediction)],'With probability of ',np.max(prediction),'%')
    


In [80]:
example_review = "Its amazing"

sentiment_predict(example_review)



[[70, 180]]
1/1 [==============================] - 0s 30ms/step
[[0.04236572 0.17972253 0.9840737 ]]
The sentiment is positive With probability of  0.9840737 %
